# 1)Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news
# Для парсинга использовать xpath. Структура данных должна содержать:
# название источника,
# наименование новости,
# ссылку на новость,
# дата публикации

# 2)Сложить все новости в БД

from SAV to Love

In [38]:
from pymongo import MongoClient
from lxml import html
import requests
from pprint import pprint
from datetime import datetime, date, time
import time
import pandas as pd
import numpy as np
import re

In [22]:
#эмулируем браузер/emulate browser
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}


In [23]:
#create a dictionary where we will combine all the data from different sites/создадим словарь куда будем объединять все данные с разных сайтов
dict_for_allnews = []

In [24]:
#take general link for url / получим ссылки на главные страницы
link_yandex = 'https://yandex.ru'
link_lenta = 'https://lenta.ru'
link_mailru= 'https://news.mail.ru'

In [25]:
#we will form requests / сформируем запросы
response_yandex = requests.get(link_yandex+'/news').text
response_lenta = requests.get(link_lenta).text
response_mail = requests.get(link_mailru).text


In [26]:
#take html
root_yandex = html.fromstring(response_yandex)
root_lenta = html.fromstring(response_lenta)
root_mail = html.fromstring(response_mail)

In [27]:
#take href for link news / создаем список ссылок на новости
links_y = root_yandex.xpath("//div[@class='page-content']//table[@class='stories-set__items']//h2[@class='story__title']\
                   | //div[@class='story__content']//h2[@class='story__title']")
links_l = root_lenta.xpath("//div[contains(@class,'item')]//a[contains(@href,'news' )\
                    and not(contains(@class,'topic-title-pic__link js-dh'))]/@href")
links_m = root_mail.xpath("//div[contains(@class,'daynews ')]//div[contains(@class,'daynews__item')]/a[@href]/@href\
                   | //div[contains(@class,'daynews ')]/..//li[contains(@class,'list__item')]/a[@href]/@href\
                   | //div[contains(@class,'cols__column_small_percent-50')]//a[@class='newsitem__title link-holder']/@href\
                   | //div[contains(@class,'cols__column_small_percent-50')]//li[contains(@class,'list__item')]//a[@href]/@href")

In [40]:
# fill out dictionaries with news on sites/ заполняем словари с данными новостей по сайтам
#yandex.ru
for item in links_y:
    dlink = item.xpath("./a[@href]/@href")[0]
    if dlink[:4] == 'http':  
        link_y = dlink
    else:
        link_y = link_yandex + dlink
    dict_yandex = {}
    dict_yandex['link'] = link_y
    dict_yandex['name'] = item.xpath("./a[@href]/text()")[0]
    txt = item.xpath("./../../..//div[@class='story__date']/text()")[0]
    dtt = txt.split()[-1]
    now = datetime.now(tz=None)
    if txt.split()[-2:-1][0] == 'в':  
        src = (' '.join(txt.split()[0:-3]))
        dt = '{:%Y-%m-}{}'.format(now, now.day - 1) + " " + dtt + ':00'
    else:
        src = (' '.join(txt.split()[0:-1]))
        dt = '{:%Y-%m-%d}'.format(now) + " " + dtt + ':00'
    dict_yandex['time'] = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')  
    dict_yandex['source'] = src
    dict_for_allnews.append(dict_yandex)
#lenta.ru
for item in links_l:
    if item[:4] == 'http': 
        pass
    else:
        link_l = link_lenta + item
    #print(href)
    dict_lenta = {}
    resp1 = requests.get(link_l).text
    article = html.fromstring(resp1) 
    head1 = article.xpath("//div[@class = 'b-topic__header js-topic__header']/h1/text()")
    dict_lenta['name'] = head1[0]
    dt = article.xpath("//div[@class = 'b-topic__header js-topic__header']//time[@datetime]/@datetime")
    dt1 = dt[0].replace('T',' ').split('+')[0]
    dict_lenta['time'] = datetime.strptime(dt1, '%Y-%m-%d %H:%M:%S')
    dict_lenta['link'] = link_l
    dict_lenta['source'] = 'lenta.ru'
    dict_for_allnews.append(dict_lenta)
#mail.ru
for item in links_m:
    if item[:4] == 'http':
        link_m = item
    else:
        link_m = link_mailru + item
    dict_mail = {}
    resp1 = requests.get(link_m).text
    article = html.fromstring(resp1) 
    head1 = article.xpath("//div[contains(@class,'article')]//span[@class='hdr__text']/h1[@class='hdr__inner']/text()")
    if head1: 
        dict_mail['name'] = head1[0]
        dt = article.xpath("//div[contains(@class,'article')]//div[@class='breadcrumbs breadcrumbs_article js-ago-wrapper']//span[@datetime]/@datetime")
        dt1 = dt[0].replace('T',' ').split('+')[0]
        dtt = (datetime.strptime(dt1, '%Y-%m-%d %H:%M:%S')) 
        dict_mail['time'] = dtt
        dict_mail['link'] = link_m
        source = article.xpath("//div[contains(@class,'article')]//span[ @class='breadcrumbs__item'][2]//a[@href]/span/text()")
        dict_mail['source'] = source[0]
        dict_for_allnews.append(dict_mail)
    else:
        pass
    # go around the avtoban/ обходим автобан
    time.sleep(1/2)

# 2)Сложить все новости в БД¶

In [42]:
#go to mongo
client = MongoClient('localhost', 27017)
db = client['parsers_news']
collection = db.mynews
collection.insert_many(dict_for_allnews)

In [43]:
#look at the date frame of the pandas and save in csv
df = pd.DataFrame(dict_for_allnews)
df.to_csv("parsers_news.csv", sep=";", index = False)
df.head(15)

,_id,link,name,source,time
0,5e9c6face07c0197bc24df3f,https://yandex.ru/news/story/Peskov_dopustil_v...,Песков допустил выход на пик по коронавирусу н...,РИА Новости,2020-04-19 17:36:00
1,5e9c6face07c0197bc24df40,https://yandex.ru/news/story/Tank_T-14_Armata_...,Танк Т-14 «Армата» испытали в Сирии,Российская газета,2020-04-19 17:55:00
2,5e9c6face07c0197bc24df41,https://yandex.ru/news/story/V_FNB_k_koncu_god...,В ФНБ к концу года останется семь триллионов р...,РИА Новости,2020-04-19 17:53:00
3,5e9c6face07c0197bc24df42,https://yandex.ru/news/story/Putin_prokommenti...,Путин прокомментировал страхи по поводу возрож...,РИА Новости,2020-04-19 17:50:00
4,5e9c6face07c0197bc24df43,https://yandex.ru/news/story/Minpromtorg_ne_vy...,Минпромторг не выявил серьезных изменений цен ...,Газета.Ru,2020-04-19 17:51:00
5,5e9c6face07c0197bc24df44,https://yandex.ru/news/story/Pochti_85_novykh_...,Почти 85% новых заболевших COVID-19 оказались ...,Газета.Ru,2020-04-19 12:51:00
6,5e9c6face07c0197bc24df45,https://yandex.ru/news/story/V_Moskve_ustanovi...,В Москве установили выплаты донорам крови с ан...,RT на русском,2020-04-19 17:57:00
7,5e9c6face07c0197bc24df46,https://yandex.ru/news/story/CHislo_bessimptom...,Число бессимптомных носителей коронавируса в М...,ТАСС,2020-04-19 16:56:00
8,5e9c6face07c0197bc24df47,https://yandex.ru/news/story/Vlasti_Moskvy_oby...,Власти Москвы объяснили обновленные правила ис...,РБК,2020-04-19 14:41:00
9,5e9c6face07c0197bc24df48,https://yandex.ru/news/story/Okhlobystina_zame...,Охлобыстина заметили нарушающим режим изоляции...,Вести.Ru,2020-04-19 17:28:00
